# Relazioni tra le tabelle con SQLAlchemy

Vediamo come definire delle relazioni tra due tabelle in modo da accedere direttamente ai record collegati senza dover eseguire delle query con il join.

## Argomenti `backref` e `back_populates`

In Flask-SQLAlchemy, le dichiarazioni `backref` e `back_populates` vengono utilizzate per definire relazioni bidirezionali tra i modelli, ma lo fanno in modi leggermente diversi.

### Argomento `backref`

Quando usi `backref`, stai creando un riferimento inverso (back-reference) in modo automatico. Questo significa che, oltre a definire la relazione da un modello all'altro, stai anche creando una proprietà sull'altro modello che ti permette di accedere alla relazione inversa. Ecco un esempio:

```python
user = db.relationship('Utente', backref=db.backref('messaggi', lazy=True))
```

In questo caso:
- `user` è un attributo nel modello che sta definendo la relazione verso il modello `Utente`.
- `backref='messaggi'` crea automaticamente una proprietà `messaggi` nel modello `Utente` che ti permette di accedere ai messaggi associati a un utente specifico.

Il parametro `lazy=True` specifica come i dati della relazione devono essere caricati. Con `lazy=True`, i dati vengono caricati solo quando acceduti (lazy loading).

### Argomento `back_populates`

Quando usi `back_populates`, devi definire la relazione bidirezionale esplicitamente in entrambi i modelli coinvolti. Questo richiede di specificare il nome dell'attributo che definisce la relazione inversa sull'altro modello. Ecco un esempio:

Nel modello `Messaggio`:

```python
user = db.relationship('Utente', back_populates='messaggi')
```

E nel modello `Utente`:

```python
messaggi = db.relationship('Messaggio', back_populates='user')
```

In questo caso:
- `user` è un attributo nel modello `Messaggio` che definisce la relazione verso il modello `Utente`.
- `back_populates='messaggi'` dice a SQLAlchemy che l'attributo `messaggi` nel modello `Utente` è la controparte di questa relazione.
- Analogamente, `messaggi` nel modello `Utente` ha `back_populates='user'` per indicare la relazione inversa.

### Riassumendo

- **`backref`**: Crea automaticamente una proprietà inversa nel modello collegato senza bisogno di definire esplicitamente la relazione in entrambi i modelli. È più conveniente e richiede meno codice.
- **`back_populates`**: Richiede la definizione esplicita della relazione in entrambi i modelli coinvolti. È più chiaro e può essere preferibile quando si vuole mantenere un maggiore controllo sulle relazioni tra i modelli.

Entrambi i metodi creano relazioni bidirezionali, ma `backref` è più automatico, mentre `back_populates` richiede una definizione esplicita in entrambi i modelli.

## Argomenti `lazy` e `uselist`

Gli argomenti `lazy` e `uselist` in SQLAlchemy (e quindi in Flask-SQLAlchemy) controllano rispettivamente come vengono caricati i dati delle relazioni e come viene gestito il tipo di relazione. Vediamo nel dettaglio cosa fanno.

### Argomento `lazy`

L'argomento `lazy` determina come vengono caricati i dati delle relazioni. Può avere diversi valori che influenzano il comportamento di caricamento. Ecco i valori più comuni:

- **`select` (predefinito)**: Carica i dati della relazione quando l'attributo viene accesso per la prima volta (lazy loading). Ad esempio, se accedi a `utente.messaggi`, SQLAlchemy esegue una query per caricare i messaggi associati a quell'utente.
  
- **`joined`**: Carica i dati della relazione tramite un'operazione di join alla query principale. Questo è utile quando sai che avrai bisogno dei dati correlati insieme al modello principale, riducendo il numero di query al database.

- **`subquery`**: Carica i dati della relazione tramite una subquery. Questo può essere utile per migliorare le prestazioni in alcune situazioni complesse.

- **`dynamic`**: Restituisce un oggetto query piuttosto che gli oggetti stessi. Questo è utile quando vuoi eseguire ulteriori filtri o operazioni sui dati correlati prima di caricarli.

- **`noload`**: Non carica mai i dati della relazione. Utilizzato principalmente per evitare il caricamento automatico dei dati quando non sono necessari.

- **`selectin`**: Simile a `subquery`, ma utilizza una tecnica di `SELECT IN` per caricare i dati in modo più efficiente.

Esempio di utilizzo di `lazy`:

```python
class Utente(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nome = db.Column(db.String(50), nullable=False)
    messaggi = db.relationship('Messaggio', backref='utente', lazy='joined')
```

### Argomento `uselist`

L'argomento `uselist` specifica se la relazione restituisce una lista di oggetti o un singolo oggetto. Questo è particolarmente rilevante nelle relazioni uno-a-uno. I valori possibili sono:

- **`True` (predefinito)**: La relazione restituisce una lista di oggetti. Questo è il comportamento predefinito per le relazioni uno-a-molti e molti-a-molti.

- **`False`**: La relazione restituisce un singolo oggetto. Questo è utile per le relazioni uno-a-uno, dove si vuole che l'attributo della relazione contenga un solo oggetto anziché una lista.

Esempio di utilizzo di `uselist`:

```python
class Indirizzo(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    via = db.Column(db.String(100), nullable=False)
    utente_id = db.Column(db.Integer, db.ForeignKey('utente.id'), unique=True)

class Utente(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nome = db.Column(db.String(50), nullable=False)
    indirizzo = db.relationship('Indirizzo', backref='utente', uselist=False)
```

In questo esempio, `uselist=False` specifica che un utente può avere un solo indirizzo, e la relazione `indirizzo` nel modello `Utente` restituirà un singolo oggetto `Indirizzo` anziché una lista.